In [0]:
import tensorflow as tf
from keras.datasets import mnist
import numpy as np
import pandas as pd
from keras import backend as K

In [0]:
def inference(x):
    phase_train = tf.constant(True)
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(inputs=x, filters=32,  kernel_size=[7, 7], padding="same", activation=tf.nn.relu)
    norm1 = tf.layers.batch_normalization(conv1)
    pool1 = tf.layers.max_pooling2d(inputs=norm1, pool_size=[2, 2], strides=2)

    conv2a = tf.layers.conv2d(inputs=pool1, filters=32, kernel_size=[1, 1], padding="same", activation=tf.nn.relu)
    conv2 = tf.layers.conv2d(inputs=conv2a, filters=64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
    norm2 = tf.layers.batch_normalization(conv2)
    pool2 = tf.layers.max_pooling2d(inputs=norm2, pool_size=[2, 2], strides=2)

    conv3a = tf.layers.conv2d(inputs=pool2, filters=32, kernel_size=[1, 1], padding="same", activation=tf.nn.relu)
    conv3 = tf.layers.conv2d(inputs=conv3a, filters=64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)

    conv4a = tf.layers.conv2d(inputs=conv3, filters=32, kernel_size=[1, 1], padding="same", activation=tf.nn.relu)
    conv4 = tf.layers.conv2d(inputs=conv4a, filters=64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)

    flat = tf.reshape(conv4, [-1, 7 * 7 * 64])

    fc_1 = tf.layers.dense(inputs=flat, units=128, activation=tf.nn.relu)

    embed = tf.layers.dense(inputs=fc_1, units=128)

    output = tf.nn.l2_normalize(embed, dim=1, epsilon=1e-12, name=None)

    return output

In [0]:
def loss(output, labels):
    triplet = tf.contrib.losses.metric_learning.triplet_semihard_loss(labels, output, margin=margin)
    loss = tf.reduce_mean(triplet, name='triplet')
    return loss

In [0]:
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    learning_rate = tf.train.exponential_decay(0.01, global_step, 100, 0.96, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [0]:
training_epochs = 1
display_step = 1
batch_size = 128
margin = 1.0
# input image dimensions
img_rows, img_cols = 28, 28

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
y_train shape: (60000,)


In [10]:
x_train_real = []
for i in range(int(60000)):
    img = x_train[i].reshape((28, 28))
    x_train_real.append(img.reshape((28, 28, 1)))

x_train_real = np.array(x_train_real)

x = tf.placeholder("float", [None, 28, 28, 1], name='placehold_x')
y = tf.placeholder('float', [None], name='placehold_y')

output = inference(x)
tf.identity(output, name="inference")

cost = loss(output, y)
global_step = tf.Variable(0, name='global_step', trainable=False)

train_op = training(cost, global_step)
saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
dim is deprecated, use axis instead

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [11]:
features_placeholder = tf.placeholder(x_train_real.dtype, x_train_real.shape)
labels_placeholder = tf.placeholder(y_train.dtype, y_train.shape)

training_dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
batched_dataset = training_dataset.batch(batch_size)

training_init_op = batched_dataset.make_initializable_iterator()
next_element = training_init_op.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [12]:
  sess = tf.Session()
  init_op = tf.global_variables_initializer()
  sess.run(init_op)

  with tf.device('/cpu:0'):
      with sess.as_default():

          # Training cycle
          for epoch in range(training_epochs):
              avg_cost = 0.
              total_batch_train = int(x_train.shape[0] / batch_size)
              sess.run(training_init_op.initializer, feed_dict={features_placeholder: x_train_real,
                                                                labels_placeholder: y_train})
              # Loop over all batches
              for i in range(total_batch_train):
                  # Fit training using batch data
                  batch_x, batch_y = sess.run(next_element)

                  sess.run(train_op, feed_dict={x: batch_x, y: batch_y})
                  # Compute average loss
                  avg_cost = sess.run(cost, feed_dict={x: batch_x, y: batch_y})

                  if not i % 10:
                      print('Epoch #: ', epoch, 'global step', sess.run(global_step), '  Batch #: ', i, 'loss: ',
                            avg_cost)

              saver.save(sess, "model_logs/model-checkpoint", global_step=global_step, write_meta_graph=True)

          print("Optimization Finished!")

Epoch #:  0 global step 1   Batch #:  0 loss:  0.9993465
Epoch #:  0 global step 11   Batch #:  10 loss:  0.9991608
Epoch #:  0 global step 21   Batch #:  20 loss:  0.935331
Epoch #:  0 global step 31   Batch #:  30 loss:  0.7849563
Epoch #:  0 global step 41   Batch #:  40 loss:  0.7252375
Epoch #:  0 global step 51   Batch #:  50 loss:  0.6756219
Epoch #:  0 global step 61   Batch #:  60 loss:  0.7249879
Epoch #:  0 global step 71   Batch #:  70 loss:  0.55965716
Epoch #:  0 global step 81   Batch #:  80 loss:  0.5651544
Epoch #:  0 global step 91   Batch #:  90 loss:  0.66573197
Epoch #:  0 global step 101   Batch #:  100 loss:  0.49763736
Epoch #:  0 global step 111   Batch #:  110 loss:  0.4020916
Epoch #:  0 global step 121   Batch #:  120 loss:  0.38649926
Epoch #:  0 global step 131   Batch #:  130 loss:  0.45992598
Epoch #:  0 global step 141   Batch #:  140 loss:  0.18870158
Epoch #:  0 global step 151   Batch #:  150 loss:  0.24522981
Epoch #:  0 global step 161   Batch #:  

In [13]:
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
y_train shape: (60000,)


In [0]:
x_train_real = []
for i in range(int(60000)):
    img = x_train[i].reshape((28, 28))
    img = np.uint8(img * 255)
    img = img / 255
    x_train_real.append(img.reshape((28, 28, 1)))

x_train_real = np.array(x_train_real)

checkpoint_directory = './model_logs/'
checkpoint_file = tf.train.latest_checkpoint(checkpoint_directory)

In [15]:
graph=tf.Graph()
with tf.Session(graph=graph) as sess:

    saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
    saver.restore(sess, checkpoint_file)
    # We can verify that we can access the list of operations in the graph
    for op in graph.get_operations():
        print(op.name)
    #keep_prob = graph.get_tensor_by_name('keep_prob:0')
    x = graph.get_tensor_by_name('placehold_x:0')

    # keep_prob = graph.get_tensor_by_name('keep_prob:0')  # dropout probability
    oper_restore = graph.get_tensor_by_name('inference:0')

    embed_train = []
    embed_test = []

    for i in range(x_test.shape[0]):
        number = x_test[i,:]
        #print(number)
        number = np.reshape(number, (1,28,28,1))
        prediction = sess.run(oper_restore, feed_dict={x: number})
        prediction = np.reshape(prediction,(128))
        embed_test.append(prediction)

    embed_test=np.asanyarray(embed_test)

    for i in range(x_train_real.shape[0]):
        number = x_train_real[i,:,:,:]
        #print(number)
        number = np.reshape(number, (1,28,28,1))
        prediction = sess.run(oper_restore, feed_dict={x: number})
        prediction = np.reshape(prediction,(128))
        embed_train.append(prediction)

    embed_train=np.asanyarray(embed_train)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model_logs/model-checkpoint-468
placehold_x
placehold_y
Const
Reshape/shape
Reshape
conv2d/kernel/Initializer/random_uniform/shape
conv2d/kernel/Initializer/random_uniform/min
conv2d/kernel/Initializer/random_uniform/max
conv2d/kernel/Initializer/random_uniform/RandomUniform
conv2d/kernel/Initializer/random_uniform/sub
conv2d/kernel/Initializer/random_uniform/mul
conv2d/kernel/Initializer/random_uniform
conv2d/kernel
conv2d/kernel/Assign
conv2d/kernel/read
conv2d/bias/Initializer/zeros
conv2d/bias
conv2d/bias/Assign
conv2d/bias/read
conv2d/dilation_rate
conv2d/Conv2D
conv2d/BiasAdd
conv2d/Relu
batch_normalization/gamma/Initializer/ones
batch_normalization/gamma
batch_normalization/gamma/Assign
batch_normalization/gamma/read
batch_normalization/beta/Initializer/zeros
batch_normalization/beta
batch_normalization/beta/Assign
batch_normalization/beta/read
batch

In [16]:
np.save('./embeddings_train.npy', embed_train)
np.save('./labels_train.npy', y_train)
np.save('./embeddings_test.npy', embed_test)
np.save('./labels_test.npy', y_test)

print("Saved")

Saved


In [0]:
embed_train = np.load('./embeddings_train.npy')
embed_test = np.load('./embeddings_test.npy')
y_train = np.load('./labels_train.npy')
y_test = np.load('./labels_test.npy')

In [0]:
matrix  = np.zeros((embed_train.shape[0]))

In [0]:
number_test = embed_test[5,:]
number_label = y_test[5]

In [0]:
for i in range(embed_train.shape[0]):

    dist = np.sqrt(np.sum(np.square(np.subtract(embed_train[i, :], number_test))))
    matrix[i] = dist

In [0]:
matrix_pd = pd.DataFrame(matrix, index=y_train).sort_values(by=[0])

In [24]:
print("Closest labels: ", matrix_pd.iloc[0:10], "Real label: ", number_label)

Closest labels:            0
1  0.020966
1  0.021701
1  0.024681
1  0.025115
1  0.029405
1  0.029639
1  0.029865
1  0.030077
1  0.030608
1  0.030814 Real label:  1
